In [1]:
"""XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.100000001, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,tree_method='hist', 
             validate_parameters=1, verbosity=None) """

"XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,\n             colsample_bynode=1, colsample_bytree=1, eta=0.1, gamma=0,\n             gpu_id=-1, importance_type='gain', interaction_constraints='',\n             learning_rate=0.100000001, max_delta_step=0, max_depth=3,\n             min_child_weight=1, missing=nan, monotone_constraints='()',\n             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,\n             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,tree_method='hist', \n             validate_parameters=1, verbosity=None) "

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor, StackingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV, GridSearchCV, train_test_split, cross_val_predict
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.datasets import make_regression
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
%matplotlib inline
pd.set_option('display.max_columns', None)

In [3]:
freq_train = pd.read_csv("frequency_domain_features_train.csv")
heart_train = pd.read_csv("heart_rate_non_linear_features_train.csv")
time_train = pd.read_csv("time_domain_features_train.csv")

In [4]:
freq_train.shape, heart_train.shape, time_train.shape

((369289, 12), (369289, 7), (369289, 20))

In [5]:
df_train = freq_train.merge(heart_train,how='inner',on='uuid')

In [6]:
df_train = df_train.merge(time_train,how='inner',on='uuid')

In [7]:
df_train.shape

(369289, 37)

In [8]:
# RANDOM SAMPLE OF DATAFRAME
df_train = df_train.sample(50000)

In [9]:
freq_test = pd.read_csv("frequency_domain_features_test.csv")
heart_test = pd.read_csv("heart_rate_non_linear_features_test.csv")
time_test = pd.read_csv("time_domain_features_test.csv")

In [10]:
freq_test.shape, heart_test.shape, time_test.shape

((41033, 12), (41033, 7), (41033, 19))

In [11]:
df_test = freq_test.merge(heart_test,how='inner',on='uuid')
df_test = df_test.merge(time_test,how='inner',on='uuid')

In [12]:
df_test.shape

(41033, 36)

In [13]:
import category_encoders as ce
features = ['condition']

count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(df_train[features])
df_train['ce'] = count_encoded

In [14]:
df_train.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,ce
227243,5ffdaf72-5d2d-418f-9751-fdaeab529673,1592.494397,75.956235,504.025128,24.040180,99.985089,0.075165,0.003585,0.014911,2096.594691,6705.571260,0.000149,8.639100,103.013421,2.104956,1.137015,2,time pressure,1107.470403,1103.201100,73.097192,12.213461,12.213457,5.984970,54.416730,3.800000,0.000000,0.647509,0.033719,8.716631e-07,-0.000528,0.011016,0.004115,0.004115,2.677416,0.647509,0.033719,8626
236022,628c4e1a-6915-48ff-bccc-f20e9f5da6a2,2442.261757,68.706709,1111.539817,31.270294,99.926509,0.817478,0.022998,0.073491,3554.619052,1359.718163,0.000735,12.347294,125.469179,2.170703,1.130515,2,interruption,1089.187338,1090.210200,89.148669,17.455910,17.455885,5.107076,55.475497,14.733333,0.933333,0.171538,-0.359109,-2.325199e-05,-0.001169,0.016473,0.006525,0.006525,2.524399,0.171538,-0.359109,14264
93511,4e1e672a-ef66-43e9-82d0-901409fe7dfc,2134.067058,73.122399,759.610727,26.027560,96.837364,24.808320,0.850041,3.162636,2918.486105,30.619193,0.032659,9.890699,113.301166,2.180771,1.198249,2,interruption,862.111306,868.637445,80.420707,13.982965,13.982894,5.751334,70.231736,7.333333,0.533333,-0.204737,-0.243422,-5.525214e-05,0.000649,0.016483,0.008388,0.008388,1.965137,-0.204737,-0.243422,14264
227455,11d173a1-a547-469e-9e19-e1d48ddd1ba6,819.096180,38.090727,1275.814000,59.329641,95.833206,55.471940,2.579632,4.166794,2150.382120,22.999268,0.043480,13.555470,87.241367,2.203131,1.293001,2,no stress,807.754488,811.160760,62.429187,19.163951,19.163934,3.257636,74.737845,21.133333,0.000000,-0.215995,-0.190797,-5.317312e-05,-0.001977,0.023659,0.013682,0.013682,1.729256,-0.215995,-0.190797,27110
343182,5b3b64e5-a5e7-422d-a381-77e9882a2b5a,533.747458,38.917296,788.378766,57.483309,94.107342,49.365395,3.599395,5.892658,1371.491619,15.970272,0.062616,8.176673,113.996682,1.816047,1.203152,2,time pressure,595.289362,571.344655,80.814916,11.560814,11.559704,6.990417,102.426748,4.866667,0.333333,1.191132,1.313563,-2.463768e-04,0.000271,0.017814,0.009993,0.009993,1.782724,1.191132,1.313563,8626


In [15]:
import category_encoders as ce
features = ['condition']

count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(df_test[features])
df_test['ce'] = count_encoded

In [16]:
df_test.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,ce
0,62b75db5-bc40-4c8f-9166-daf0efcab4c2,1868.532278,76.511189,570.643114,23.366245,99.478197,2.993254,0.122565,0.521803,2442.168645,190.643094,0.005245,8.347898,115.862444,2.209659,1.100715,2,time pressure,934.665288,939.03173,82.139495,11.801781,11.801772,6.959924,3.933333,0.133333,-0.680262,-0.233075,0.000016,0.000288,0.012933,0.004578,0.004578,2.825038,-0.680262,-0.233075,7093
1,a99549ad-3eb6-4413-bc90-9053e7f7e684,568.742845,26.301350,1553.971621,71.862973,97.509212,39.694850,1.835677,2.490788,2162.409316,39.147940,0.025544,14.542096,77.118903,2.186132,1.290615,2,no stress,817.062380,816.33879,55.492332,20.558810,20.558768,2.699200,24.600000,0.533333,-0.034454,-0.051689,0.000040,-0.002749,0.025148,0.013921,0.013921,1.806517,-0.034454,-0.051689,22158
2,cb573d3a-c767-4556-b32e-ad8c08ded214,2101.871207,75.836461,655.175895,23.639042,97.829386,14.536877,0.524497,2.170614,2771.583978,45.069921,0.022188,9.799336,125.044377,2.051571,1.226663,2,no stress,876.762022,894.19889,88.690820,13.853737,13.853730,6.401942,7.066667,0.533333,-0.206953,-0.589940,0.000010,-0.000172,0.015533,0.008149,0.008149,1.906180,-0.206953,-0.589940,22158
3,47a0c6de-2aef-4ac3-997d-252fa6fd07f1,5757.544433,90.562305,592.913021,9.326123,98.817806,7.093235,0.111572,1.182194,6357.550689,83.588517,0.011963,11.639185,302.029812,2.080910,1.085143,2,time pressure,1038.640693,998.91429,213.725850,16.457194,16.454801,12.986774,10.800000,1.866667,-0.820407,0.487198,-0.000238,-0.000464,0.016882,0.007587,0.007587,2.225116,-0.820407,0.487198,7093
4,de3fd54f-c74e-4fe8-bf2a-7a127f68b312,964.696325,70.256575,374.939530,27.305968,91.805057,33.468834,2.437457,8.194943,1373.104689,11.202647,0.089265,7.266567,72.579248,2.068728,1.252547,2,interruption,774.548508,778.90508,51.577855,10.273114,10.273049,5.020664,2.000000,0.066667,1.738453,-0.005082,0.000054,-0.000006,0.013479,0.007811,0.007811,1.725606,1.738453,-0.005082,11782


In [17]:
df_train['pNN50s'] = df_train['pNN50'].astype(str)

In [18]:
df_test['pNN50s'] = df_test['pNN50'].astype(str)

In [19]:
train_interactions = df_train['condition'] + '-' + df_train['pNN50s']

In [20]:
test_interactions = df_test['condition'] + '-' + df_test['pNN50s']

In [21]:
df_train['interactions'] = train_interactions

In [22]:
df_test['interactions'] = test_interactions

In [23]:
df_train.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,ce,pNN50s,interactions
227243,5ffdaf72-5d2d-418f-9751-fdaeab529673,1592.494397,75.956235,504.025128,24.040180,99.985089,0.075165,0.003585,0.014911,2096.594691,6705.571260,0.000149,8.639100,103.013421,2.104956,1.137015,2,time pressure,1107.470403,1103.201100,73.097192,12.213461,12.213457,5.984970,54.416730,3.800000,0.000000,0.647509,0.033719,8.716631e-07,-0.000528,0.011016,0.004115,0.004115,2.677416,0.647509,0.033719,8626,0.0,time pressure-0.0
236022,628c4e1a-6915-48ff-bccc-f20e9f5da6a2,2442.261757,68.706709,1111.539817,31.270294,99.926509,0.817478,0.022998,0.073491,3554.619052,1359.718163,0.000735,12.347294,125.469179,2.170703,1.130515,2,interruption,1089.187338,1090.210200,89.148669,17.455910,17.455885,5.107076,55.475497,14.733333,0.933333,0.171538,-0.359109,-2.325199e-05,-0.001169,0.016473,0.006525,0.006525,2.524399,0.171538,-0.359109,14264,0.9333333333333332,interruption-0.9333333333333332
93511,4e1e672a-ef66-43e9-82d0-901409fe7dfc,2134.067058,73.122399,759.610727,26.027560,96.837364,24.808320,0.850041,3.162636,2918.486105,30.619193,0.032659,9.890699,113.301166,2.180771,1.198249,2,interruption,862.111306,868.637445,80.420707,13.982965,13.982894,5.751334,70.231736,7.333333,0.533333,-0.204737,-0.243422,-5.525214e-05,0.000649,0.016483,0.008388,0.008388,1.965137,-0.204737,-0.243422,14264,0.5333333333333333,interruption-0.5333333333333333
227455,11d173a1-a547-469e-9e19-e1d48ddd1ba6,819.096180,38.090727,1275.814000,59.329641,95.833206,55.471940,2.579632,4.166794,2150.382120,22.999268,0.043480,13.555470,87.241367,2.203131,1.293001,2,no stress,807.754488,811.160760,62.429187,19.163951,19.163934,3.257636,74.737845,21.133333,0.000000,-0.215995,-0.190797,-5.317312e-05,-0.001977,0.023659,0.013682,0.013682,1.729256,-0.215995,-0.190797,27110,0.0,no stress-0.0
343182,5b3b64e5-a5e7-422d-a381-77e9882a2b5a,533.747458,38.917296,788.378766,57.483309,94.107342,49.365395,3.599395,5.892658,1371.491619,15.970272,0.062616,8.176673,113.996682,1.816047,1.203152,2,time pressure,595.289362,571.344655,80.814916,11.560814,11.559704,6.990417,102.426748,4.866667,0.333333,1.191132,1.313563,-2.463768e-04,0.000271,0.017814,0.009993,0.009993,1.782724,1.191132,1.313563,8626,0.3333333333333333,time pressure-0.3333333333333333


In [24]:
df_test.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,ce,pNN50s,interactions
0,62b75db5-bc40-4c8f-9166-daf0efcab4c2,1868.532278,76.511189,570.643114,23.366245,99.478197,2.993254,0.122565,0.521803,2442.168645,190.643094,0.005245,8.347898,115.862444,2.209659,1.100715,2,time pressure,934.665288,939.03173,82.139495,11.801781,11.801772,6.959924,3.933333,0.133333,-0.680262,-0.233075,0.000016,0.000288,0.012933,0.004578,0.004578,2.825038,-0.680262,-0.233075,7093,0.1333333333333333,time pressure-0.1333333333333333
1,a99549ad-3eb6-4413-bc90-9053e7f7e684,568.742845,26.301350,1553.971621,71.862973,97.509212,39.694850,1.835677,2.490788,2162.409316,39.147940,0.025544,14.542096,77.118903,2.186132,1.290615,2,no stress,817.062380,816.33879,55.492332,20.558810,20.558768,2.699200,24.600000,0.533333,-0.034454,-0.051689,0.000040,-0.002749,0.025148,0.013921,0.013921,1.806517,-0.034454,-0.051689,22158,0.5333333333333333,no stress-0.5333333333333333
2,cb573d3a-c767-4556-b32e-ad8c08ded214,2101.871207,75.836461,655.175895,23.639042,97.829386,14.536877,0.524497,2.170614,2771.583978,45.069921,0.022188,9.799336,125.044377,2.051571,1.226663,2,no stress,876.762022,894.19889,88.690820,13.853737,13.853730,6.401942,7.066667,0.533333,-0.206953,-0.589940,0.000010,-0.000172,0.015533,0.008149,0.008149,1.906180,-0.206953,-0.589940,22158,0.5333333333333333,no stress-0.5333333333333333
3,47a0c6de-2aef-4ac3-997d-252fa6fd07f1,5757.544433,90.562305,592.913021,9.326123,98.817806,7.093235,0.111572,1.182194,6357.550689,83.588517,0.011963,11.639185,302.029812,2.080910,1.085143,2,time pressure,1038.640693,998.91429,213.725850,16.457194,16.454801,12.986774,10.800000,1.866667,-0.820407,0.487198,-0.000238,-0.000464,0.016882,0.007587,0.007587,2.225116,-0.820407,0.487198,7093,1.866666666666667,time pressure-1.866666666666667
4,de3fd54f-c74e-4fe8-bf2a-7a127f68b312,964.696325,70.256575,374.939530,27.305968,91.805057,33.468834,2.437457,8.194943,1373.104689,11.202647,0.089265,7.266567,72.579248,2.068728,1.252547,2,interruption,774.548508,778.90508,51.577855,10.273114,10.273049,5.020664,2.000000,0.066667,1.738453,-0.005082,0.000054,-0.000006,0.013479,0.007811,0.007811,1.725606,1.738453,-0.005082,11782,0.06666666666666668,interruption-0.06666666666666668


In [25]:
label_enc = LabelEncoder()
df_train = df_train.assign(condition_pNN50s= label_enc.fit_transform(train_interactions))

In [26]:
label_enc = LabelEncoder()
df_test = df_test.assign(condition_pNN50s= label_enc.fit_transform(test_interactions))

In [27]:
df_train.head()

,uuid,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,SD1,SD2,sampen,higuci,datasetId,condition,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,ce,pNN50s,interactions,condition_pNN50s
227243,5ffdaf72-5d2d-418f-9751-fdaeab529673,1592.494397,75.956235,504.025128,24.040180,99.985089,0.075165,0.003585,0.014911,2096.594691,6705.571260,0.000149,8.639100,103.013421,2.104956,1.137015,2,time pressure,1107.470403,1103.201100,73.097192,12.213461,12.213457,5.984970,54.416730,3.800000,0.000000,0.647509,0.033719,8.716631e-07,-0.000528,0.011016,0.004115,0.004115,2.677416,0.647509,0.033719,8626,0.0,time pressure-0.0,141
236022,628c4e1a-6915-48ff-bccc-f20e9f5da6a2,2442.261757,68.706709,1111.539817,31.270294,99.926509,0.817478,0.022998,0.073491,3554.619052,1359.718163,0.000735,12.347294,125.469179,2.170703,1.130515,2,interruption,1089.187338,1090.210200,89.148669,17.455910,17.455885,5.107076,55.475497,14.733333,0.933333,0.171538,-0.359109,-2.325199e-05,-0.001169,0.016473,0.006525,0.006525,2.524399,0.171538,-0.359109,14264,0.9333333333333332,interruption-0.9333333333333332,14
93511,4e1e672a-ef66-43e9-82d0-901409fe7dfc,2134.067058,73.122399,759.610727,26.027560,96.837364,24.808320,0.850041,3.162636,2918.486105,30.619193,0.032659,9.890699,113.301166,2.180771,1.198249,2,interruption,862.111306,868.637445,80.420707,13.982965,13.982894,5.751334,70.231736,7.333333,0.533333,-0.204737,-0.243422,-5.525214e-05,0.000649,0.016483,0.008388,0.008388,1.965137,-0.204737,-0.243422,14264,0.5333333333333333,interruption-0.5333333333333333,8
227455,11d173a1-a547-469e-9e19-e1d48ddd1ba6,819.096180,38.090727,1275.814000,59.329641,95.833206,55.471940,2.579632,4.166794,2150.382120,22.999268,0.043480,13.555470,87.241367,2.203131,1.293001,2,no stress,807.754488,811.160760,62.429187,19.163951,19.163934,3.257636,74.737845,21.133333,0.000000,-0.215995,-0.190797,-5.317312e-05,-0.001977,0.023659,0.013682,0.013682,1.729256,-0.215995,-0.190797,27110,0.0,no stress-0.0,79
343182,5b3b64e5-a5e7-422d-a381-77e9882a2b5a,533.747458,38.917296,788.378766,57.483309,94.107342,49.365395,3.599395,5.892658,1371.491619,15.970272,0.062616,8.176673,113.996682,1.816047,1.203152,2,time pressure,595.289362,571.344655,80.814916,11.560814,11.559704,6.990417,102.426748,4.866667,0.333333,1.191132,1.313563,-2.463768e-04,0.000271,0.017814,0.009993,0.009993,1.782724,1.191132,1.313563,8626,0.3333333333333333,time pressure-0.3333333333333333,146


In [28]:
conditions = [
    (df_train['condition'] == "no stress"),
    (df_train['condition'] == "interruption condition"),
    (df_train['condition'] == "time pressure" )
    ]
values = [1,0,-1]
df_train['condition2'] = np.select(conditions, values)

In [29]:
conditions = [
    (df_test['condition'] == "no stress"),
    (df_test['condition'] == "interruption condition"),
    (df_test['condition'] == "time pressure" )
    ]
values = [1,0,-1]
df_test['condition2'] = np.select(conditions, values)

In [30]:
df_train.columns

Index(['uuid', 'VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT',
       'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'SD1', 'SD2', 'sampen', 'higuci',
       'datasetId', 'condition', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD',
       'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW',
       'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR',
       'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'ce',
       'pNN50s', 'interactions', 'condition_pNN50s', 'condition2'],
      dtype='object')

In [31]:
df_test = df_test.drop(['uuid', 'datasetId', 'condition', 'interactions', 'SD2', 'SDRR_RMSSD_REL_RR',
                        'SDRR_REL_RR', 'MEAN_REL_RR', 'condition2', 'RMSSD_REL_RR', 'MEDIAN_REL_RR',
                        'RMSSD', 'LF_PCT', 'SKEW_REL_RR', 'HF', 'LF_NU', 'HF_NU', 'SD1', 'KURT_REL_RR'], axis = 1)


In [32]:
df_train = df_train.drop(['uuid', 'datasetId', 'condition', 'interactions', 'SD2', 'SDRR_RMSSD_REL_RR',
                        'SDRR_REL_RR', 'MEAN_REL_RR', 'condition2', 'RMSSD_REL_RR', 'MEDIAN_REL_RR',
                        'RMSSD', 'LF_PCT', 'SKEW_REL_RR', 'HF', 'LF_NU', 'HF_NU', 'SD1', 'KURT_REL_RR'], axis=1)


In [33]:
df_train.shape, df_test.shape

((50000, 23), (41033, 22))

In [34]:
X = df_train.drop("HR",axis=1)
y = df_train["HR"]

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.1, random_state = 1)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 1)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.11111, random_state = 1)

In [36]:
#X_train.shape, X_test.shape, X_val.shape 
X_train.shape, X_val.shape

((45000, 22), (5000, 22))

In [37]:
#y_train.shape, y_test.shape, y_val.shape
y_train.shape, y_val.shape

((45000,), (5000,))

In [38]:
sc = StandardScaler()

X_train = pd.DataFrame(sc.fit_transform(X_train),columns=X.columns)
X_val = pd.DataFrame(sc.transform(X_val),columns=X.columns)
#X_test = pd.DataFrame(sc.transform(X_test),columns=X.columns)

X_train.head()

,VLF,VLF_PCT,LF,HF_PCT,TP,LF_HF,HF_LF,sampen,higuci,MEAN_RR,MEDIAN_RR,SDRR,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,SDSD_REL_RR,ce,pNN50s,condition_pNN50s
0,1.877130,1.230995,0.059976,-0.824064,1.774699,-0.064771,-0.765833,-0.250569,-0.745407,0.912164,0.860192,0.710391,0.876197,0.276092,0.412486,1.357359,0.692954,1.155762,0.026713,-1.514757,1.357359,1.906263
1,3.479152,1.656605,-0.411988,-0.786207,3.150407,-0.231060,-0.381853,0.521856,-1.829335,2.000502,2.098752,2.445929,0.902556,1.647310,0.404359,1.627383,-0.608762,-0.085589,-0.185310,-1.514757,1.627383,1.987425
2,0.122781,1.098109,-0.789070,-0.659949,-0.134515,-0.220672,-0.441091,-0.010659,-0.272463,0.425378,0.505051,-0.227830,-0.643395,0.008074,-0.627857,-0.870340,0.207907,-0.776923,-0.808321,0.891079,-0.870340,-0.021340
3,1.578435,1.537892,-0.707657,-0.746589,1.265566,-0.235169,-0.354715,-2.794317,-1.553602,0.583437,-0.824688,3.392352,-0.443663,4.015851,-0.319005,-0.262786,-0.730852,1.192964,-0.624617,0.891079,-0.262786,0.161275
4,0.152120,-0.174156,0.872057,0.594759,0.440768,-0.288389,0.521886,-0.141284,-0.718811,-1.733615,-1.487712,-0.137560,0.106938,-0.192361,-0.180835,0.952322,-0.198316,-0.829707,1.484357,0.891079,0.952322,0.526505


In [39]:
# lasso regression
lasso = Lasso(alpha = 0.0)

# fit lasso on training data
lasso.fit(X_train, y_train)

# make prediciton on test data
y_pred_lasso = lasso.predict(X_val)

In [40]:
# calculate preformance measures lasso regression
rmse_lasso = np.sqrt(mean_squared_error(y_val, y_pred_lasso))
mse_lasso = mean_squared_error(y_val, y_pred_lasso)
mae_lasso = mean_absolute_error(y_val, y_pred_lasso)
r2_lasso = r2_score(y_val, y_pred_lasso)
adjr2_lasso = 1-(1-r2_lasso)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)


In [41]:
resultsDf2 = pd.DataFrame({'Method':['Lasso Regression'], 'RMSE': rmse_lasso , 'MSE': mse_lasso , 'MAE': mae_lasso ,"R2": r2_lasso, 'Adj_R2': adjr2_lasso})
resultsDf2

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Lasso Regression,1.475522,2.177165,0.969984,0.979673,0.979583


In [42]:
# ridge regression
ridge = Ridge(alpha = 1.95)

# fit ridge on training data
ridge.fit(X_train, y_train)

# make prediciton on test data
y_pred_ridge = ridge.predict(X_val)

In [43]:
# calculate preformance measures ridge regression
rmse_ridge = np.sqrt(mean_squared_error(y_val, y_pred_ridge))
mse_ridge = mean_squared_error(y_val, y_pred_ridge)
mae_ridge = mean_absolute_error(y_val, y_pred_ridge)
r2_ridge = r2_score(y_val, y_pred_ridge)
adjr2_ridge = 1-(1-r2_ridge)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)

resultsDf3 = pd.DataFrame({'Method':['Ridge Regression'], 'RMSE': rmse_ridge , 'MSE': mse_ridge , 'MAE': mae_ridge ,"R2": r2_ridge, 'Adj_R2': adjr2_ridge})
resultsDf3

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Ridge Regression,1.475232,2.176309,0.970181,0.979681,0.979591


In [44]:
linear = pd.concat([resultsDf2, resultsDf3],axis=0)
linear

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Lasso Regression,1.475522,2.177165,0.969984,0.979673,0.979583
0,Ridge Regression,1.475232,2.176309,0.970181,0.979681,0.979591


In [45]:
rf=RandomForestRegressor(random_state=1, n_estimators=20, max_depth=50)
rf.fit(X_train,y_train)
#rf.score(X_train,y_train)

#this line doesn't make sense to me
y_pred_rf_train=rf.predict(X_train)
#

y_pred_rf_val=rf.predict(X_val)
print('Train R2 score: ', r2_score(y_train,y_pred_rf_train))
print('Validation R2 score: ', r2_score(y_val,y_pred_rf_val))

Train R2 score:  0.9999712977837255
Validation R2 score:  0.9998733311158854


In [46]:
rmse_rf = np.sqrt(mean_squared_error(y_val, y_pred_rf_val))
mse_rf = mean_squared_error(y_val, y_pred_rf_val)
mae_rf = mean_absolute_error(y_val, y_pred_rf_val)
r2_rf = r2_score(y_val, y_pred_rf_val)
adjr2_rf = 1-(1-r2_rf)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)

resultsDf1 = pd.DataFrame({'Method':['Random Forest'], 'RMSE': rmse_rf , 'MSE': mse_rf , 'MAE': mae_rf ,"R2": r2_rf, 'Adj_R2': adjr2_rf})
resultsDf1

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Random Forest,0.116477,0.013567,0.039649,0.999873,0.999873


In [47]:
#This block doesn't make sense to me

rmse_rf = np.sqrt(mean_squared_error(y_train, y_pred_rf_train))
mse_rf = mean_squared_error(y_train, y_pred_rf_train)
mae_rf = mean_absolute_error(y_train, y_pred_rf_train)
r2_rf = r2_score(y_train, y_pred_rf_train)
adjr2_rf = 1-(1-r2_rf)*(len(y_train)-1)/(len(y_train)-len(X_train.columns)-1)

resultsDf1 = pd.DataFrame({'Method':['Random Forest'], 'RMSE': rmse_rf , 'MSE': mse_rf , 'MAE': mae_rf ,"R2": r2_rf, 'Adj_R2': adjr2_rf})
resultsDf1
#

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Random Forest,0.055343,0.003063,0.017776,0.999971,0.999971


In [48]:
# Gradient Boost
gb=GradientBoostingRegressor(random_state=1,n_estimators=50 ,max_depth=50,learning_rate=.1)
gb.fit(X_train,y_train)
#gb.score(X_train,y_train)
y_pred_gbtr=gb.predict(X_train)
y_pred_gbval=gb.predict(X_val)
print('Train R2 score: ',r2_score(y_train,y_pred_gbtr))
print('Test R2 score: ',r2_score(y_val,y_pred_gbval))

Train R2 score:  0.9999734386011124
Test R2 score:  0.9996771713959796


In [49]:
rmse_gb = np.sqrt(mean_squared_error(y_val, y_pred_gbval))
mse_gb = mean_squared_error(y_val, y_pred_gbval)
mae_gb = mean_absolute_error(y_val, y_pred_gbval)
r2_gb = r2_score(y_val, y_pred_gbval)
adjr2_gb = 1-(1-r2_gb)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)


resultsDf2 = pd.DataFrame({'Method':['Gradient Boost'], 'RMSE': rmse_gb , 'MSE': mse_gb , 'MAE': mae_gb ,"R2": r2_gb, 'Adj_R2': adjr2_gb})
resultsDf2

,Method,RMSE,MSE,MAE,R2,Adj_R2
0,Gradient Boost,0.185948,0.034577,0.073461,0.999677,0.999676


In [50]:
#AdaBoostRegressor
ab = AdaBoostRegressor(n_estimators = 15, random_state = 1)
ab.fit(X_train, y_train)
ab_pred = ab.predict(X_val)

print('AdaBoost Regressor:')
print('')
print('Training Score:', ab.score(X_train, y_train))
print('Validate Score:', ab.score(X_val, y_val))



rmse_ab = np.sqrt(mean_squared_error(y_val, ab_pred))
mse_ab = mean_squared_error(y_val, ab_pred)
mae_ab = mean_absolute_error(y_val, ab_pred)
r2_ab = r2_score(y_val, ab_pred)
adjr2_ab = 1-(1-r2_ab)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)

print('Root Mean Sq Error:', rmse_ab)
print('Mean Sq Error:', mse_ab)
print('Mean Abs Error:', mae_ab)
print('R2:', r2_ab)
print('Adj R2:', adjr2_ab)

AdaBoost Regressor:

Training Score: 0.9742920790872185
Validate Score: 0.9736090526702876
Root Mean Sq Error: 1.6812509327118579
Mean Sq Error: 2.826604698744492
Mean Abs Error: 1.332234099925604
R2: 0.9736090526702876
Adj R2: 0.9734923958808053


In [53]:
#StackingRegressor
estimators = [('bagging',BaggingRegressor()),('adboost', AdaBoostRegressor(n_estimators = 15, random_state = 1)),('gradboost',GradientBoostingRegressor(random_state=1,n_estimators=50 ,max_depth=50,learning_rate=.1))]


In [54]:
sr = StackingRegressor(estimators = estimators, final_estimator = RandomForestRegressor(random_state=1, n_estimators=20, max_depth=50))
sr.fit(X_train, y_train)
sr_pred = sr.predict(X_val)

print('Stacking Regressor:')
print('')
print('Training Score:', sr.score(X_train, y_train))
print('Validate Score:', sr.score(X_val, y_val))



rmse_sr = np.sqrt(mean_squared_error(y_val, sr_pred))
mse_sr = mean_squared_error(y_val, sr_pred)
mae_sr = mean_absolute_error(y_val, sr_pred)
r2_sr = r2_score(y_val, sr_pred)
adjr2_sr = 1-(1-r2_sr)*(len(y_val)-1)/(len(y_val)-len(X_val.columns)-1)

print('Root Mean Sq Error:', rmse_sr)
print('Mean Sq Error:', mse_sr)
print('Mean Abs Error:', mae_sr)
print('R2:', r2_sr)
print('Adj R2:', adjr2_sr)

Stacking Regressor:

Training Score: 0.9999760441227282
Validate Score: 0.9998428990081774
Root Mean Sq Error: 0.12971628784147016
Mean Sq Error: 0.016826315331371137
Mean Abs Error: 0.04609392031308721
R2: 0.9998428990081774
Adj R2: 0.999842204569395


In [55]:
# creating a new dataframe of predictions, last column is true values

#  'sr_pred': sr_pred left out
predictions = pd.DataFrame({'adaboost': ab_pred, 'y_pred_lasso': y_pred_lasso,
                            'y_pred_ridge': y_pred_ridge, 'y_pred_rf_val': y_pred_rf_val,
                            'y_pred_gbval': y_pred_gbval, 'sr_pred': sr_pred, 'y_val': y_val})     

In [56]:
predictions.head()

,adaboost,y_pred_lasso,y_pred_ridge,y_pred_rf_val,y_pred_gbval,sr_pred,y_val
201289,64.257328,63.788304,63.791830,63.238049,63.280327,63.237820,63.256769
662,83.510153,82.888846,82.884615,83.135751,83.044812,83.159489,83.149693
356149,56.132399,52.641430,52.639811,52.789225,52.879003,52.775438,52.783706
330685,63.623182,61.362124,61.355234,61.582973,61.647952,61.587704,61.580535
47460,78.776079,76.510493,76.512797,77.178147,77.171550,77.169792,77.158604


In [57]:
X2_train = predictions.drop(columns = ['y_val'])
y2_train = predictions['y_val']

In [58]:
lr2 = LinearRegression()
lr2.fit(X2_train, y2_train)

LinearRegression()

In [59]:
#
ab_train_predictions = ab.predict(X_train)
rf_train_predictions = rf.predict(X_train)
gb_train_predictions = gb.predict(X_train)
lasso_train_predictions = lasso.predict(X_train)
ridge_train_predictions = ridge.predict(X_train)
sr_train_predictions = sr.predict(X_train)

In [60]:
df_train_predictions = pd.DataFrame({'ab_train_predictions': ab_train_predictions, 'rf_train_predictions': rf_train_predictions, 
                                     'gb_train_predictions': gb_train_predictions, 'lasso_train_predictions': lasso_train_predictions,
                                     'ridge_train_predictions': ridge_train_predictions, 'sr_train_predictions': sr_train_predictions,
                                     'y_train': y_train})

In [61]:
df_train_predictions

,ab_train_predictions,rf_train_predictions,gb_train_predictions,lasso_train_predictions,ridge_train_predictions,sr_train_predictions,y_train
48206,64.788674,64.367818,64.296366,65.421605,65.414228,64.267208,64.246348
359891,58.920859,59.739664,59.824584,59.738632,59.734115,59.733620,59.751400
39560,67.927319,67.461035,67.494979,67.742391,67.747336,67.443270,67.461531
37626,77.973273,74.811091,74.765782,75.341429,75.349901,74.726566,74.770001
210265,100.132595,97.921908,97.803006,93.664969,93.661329,97.949520,97.926568
...,...,...,...,...,...,...,...
211968,61.807570,60.466732,60.524868,59.602787,59.599865,60.419263,60.455312
303746,96.083536,97.946107,97.805209,95.352786,95.334168,97.941000,97.928783
279091,77.973273,76.309978,76.298169,75.397301,75.398454,76.296801,76.310327
359841,69.231767,67.485583,67.521921,68.334703,68.342175,67.505074,67.488613


In [62]:
X2_train = df_train_predictions.drop(columns = ['y_train'])
y2_train = df_train_predictions['y_train']

In [63]:
lr2.fit(X2_train, y2_train)

LinearRegression()

In [64]:
ab_val_predictions = ab.predict(X_val)
rf_val_predictions = rf.predict(X_val)
gb_val_predictions = gb.predict(X_val)
lasso_val_predictions = lasso.predict(X_val)
ridge_val_predictions = ridge.predict(X_val)
sr_train_predictions = sr.predict(X_val)

In [65]:
df_val_predictions = pd.DataFrame({'ab_val_predictions': ab_val_predictions, 'rf_val_predictions': rf_val_predictions, 
                                     'gb_val_predictions': gb_val_predictions, 'lasso_val_predictions': lasso_val_predictions,
                                     'ridge_val_predictions': ridge_val_predictions, 'sr_train_predictions': sr_train_predictions})

In [66]:
df_val_predictions

,ab_val_predictions,rf_val_predictions,gb_val_predictions,lasso_val_predictions,ridge_val_predictions,sr_train_predictions
0,64.257328,63.238049,63.280327,63.788304,63.791830,63.237820
1,83.510153,83.135751,83.044812,82.888846,82.884615,83.159489
2,56.132399,52.789225,52.879003,52.641430,52.639811,52.775438
3,63.623182,61.582973,61.647952,61.362124,61.355234,61.587704
4,78.776079,77.178147,77.171550,76.510493,76.512797,77.169792
...,...,...,...,...,...,...
4995,65.770972,64.888722,65.051259,65.123217,65.122461,64.966555
4996,87.193189,92.186039,92.095840,93.347080,93.326581,92.194423
4997,66.839038,66.908683,66.915092,67.376961,67.367938,66.880564
4998,73.979979,74.387645,74.378489,74.405247,74.404010,74.414202


In [67]:
stacked_val_predictions = lr2.predict(df_val_predictions)

In [68]:
mean_absolute_error(y_val, stacked_val_predictions)

0.048930677860185894

In [69]:
mean_absolute_error(y_val, rf_val_predictions)

0.03964894676941249

In [70]:
#X3_train = predictions.drop(columns = ['y_val'])
#y3_train = predictions['y_val']

In [71]:
#rf2 = RandomForestRegressor()
#rf2.fit(X3_train, y3_train)

In [72]:
#
#ab_train_predictions = ab.predict(X_train)
#rf_train_predictions = rf.predict(X_train)
#gb_train_predictions = gb.predict(X_train)
#lasso_train_predictions = lasso.predict(X_train)
#ridge_train_predictions = ridge.predict(X_train)

In [73]:
#df_train_predictions = pd.DataFrame({'ab_train_predictions': ab_train_predictions, 'rf_train_predictions': rf_train_predictions, 
#                                     'gb_train_predictions': gb_train_predictions, 'lasso_train_predictions': lasso_train_predictions,
#                                     'ridge_train_predictions': ridge_train_predictions, 'y_train': y_train})

In [74]:
#df_train_predictions

In [75]:
#X3_train = df_train_predictions.drop(columns = ['y_train'])
#y3_train = df_train_predictions['y_train']

In [76]:
#rf2.fit(X3_train, y3_train)

In [77]:
#ab_val_predictions = ab.predict(X_val)
#rf_val_predictions = rf.predict(X_val)
#gb_val_predictions = gb.predict(X_val)
#lasso_val_predictions = lasso.predict(X_val)
#ridge_val_predictions = ridge.predict(X_val)

In [78]:
#df_val_predictions = pd.DataFrame({'ab_val_predictions': ab_val_predictions, 'rf_val_predictions': rf_val_predictions, 
#                                     'gb_val_predictions': gb_val_predictions, 'lasso_val_predictions': lasso_val_predictions,
#                                     'ridge_val_predictions': ridge_val_predictions})

In [79]:
#df_val_predictions

In [80]:
#stacked_val_predictions2 = rf2.predict(df_val_predictions)

In [81]:
#mean_absolute_error(y_val, stacked_val_predictions2)

In [82]:
#mean_absolute_error(y_val, rf_val_predictions)

In [83]:
X4_train = predictions.drop(columns = ['y_val'])
y4_train = predictions['y_val']

In [84]:
rf3 = RandomForestRegressor(random_state=1, n_estimators=20, max_depth=50)
rf3.fit(X4_train, y4_train)

RandomForestRegressor(max_depth=50, n_estimators=20, random_state=1)

In [85]:
X4_train = df_train_predictions.drop(columns = ['y_train'])
y4_train = df_train_predictions['y_train']

In [86]:
rf3.fit(X4_train, y4_train)

RandomForestRegressor(max_depth=50, n_estimators=20, random_state=1)

In [87]:
stacked_val_predictions3 = rf3.predict(df_val_predictions)

In [88]:
mean_absolute_error(y_val, stacked_val_predictions3)

0.04836798897547978

In [89]:
mean_absolute_error(y_val, rf_val_predictions)

0.03964894676941249